<a href="https://colab.research.google.com/github/rhodes-byu/cs180-winter25/blob/main/notebooks/08-sqlite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><p><b>After clicking the "Open in Colab" link, copy the notebook to your own Google Drive before getting started, or it will not save your work</b></p>

# Getting Started with SQLite3 in Python

This notebook provides a step-by-step guide to using SQLite3 in Python. SQLite is a lightweight, file-based database that is ideal for small to medium applications.

## What We Will Cover
- How to install SQLite3
- How to create a database and connect to it
- How to create tables and insert data
- How to query, update, and delete data
- How to use SQLite3 with Pandas

In [ ]:
import sqlite3
import pandas as pd

In [2]:
!curl -o sales.db "https://raw.githubusercontent.com/rhodes-byu/cs180-winter25/refs/heads/main/databases/sales.db"

### Establish a connection to the database

In [ ]:
conn = sqlite3.connect('sales.db')
print(type(conn))

In [ ]:
cursor = conn.cursor()
print(type(cursor))

### Executions of queries

In [ ]:
cursor.execute('SELECT * FROM Artist')

In [6]:
# Fetchall returns a list of tuples with each tuple representing a row in the table 
artists = cursor.fetchall()

# Alternative: Fetchmany(n) returns a list of n tuples
# Used for larger dataets (query batches at a time)

artists = cursor.fetchmany(10)

In [ ]:
artists[:5]

Note: the fetchall can only be called once. If you call it again, it will return an empty list


In [ ]:
cursor.fetchall()

A longer query example.

In [ ]:
query = '''
SELECT Artist.Name AS ArtistName, Album.Title AS AlbumTitle
FROM Artist
JOIN Album ON Artist.ArtistId = Album.ArtistId
'''

cursor.execute(query)

In [ ]:
artist_album = cursor.fetchall()
artist_album[:5]

### Printing all available tables

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

In [ ]:
# Close the connection
conn.close()

The `sqlite_master` table in SQLite contains information about all the tables, indexes, triggers, and views in the database. Here are the columns available in the `sqlite_master` table:

- **type**: The type of database object, which can be one of 'table', 'index', 'trigger', or 'view'.
- **name**: The name of the database object.
- **tbl_name**: The name of the table associated with the object (for indexes, triggers, and views).
- **rootpage**: The page number of the root b-tree page for tables and indexes.
- **sql**: The SQL statement that was used to create the object.


# Creating a Database

In [12]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('example.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()


## Creating a Table

Tables store data in a structured format with rows and columns. Below, we create a simple `users` table.


In [13]:
# Create a table named 'users'
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    email TEXT UNIQUE
)
''')

# Commit the changes
conn.commit()

In SQLite3, you only need to call `conn.commit()` after executing queries that modify the database. This ensures that the changes you made are permanently saved. Below are the common types of queries that require a commit:

- **INSERT Statements:** Adding new records.
- **UPDATE Statements:** Modifying existing records.
- **DELETE Statements:** Removing records.
- **DDL (Data Definition Language) Statements:** These include operations such as:
  - `CREATE TABLE`
  - `DROP TABLE`
  - `ALTER TABLE`

For queries that only retrieve data (like **SELECT** statements), calling `conn.commit()` is not necessary because they do not alter the state of the database.



## Inserting Data

We can use the `INSERT INTO` statement to add records to the table.


In [ ]:
# Insert data into the users table

try:
    cursor.execute("INSERT INTO users (name, age, email) VALUES ('Alice', 25, 'alice@example.com')")
except sqlite3.IntegrityError as e:
    print('Error inserting data:', e)

try:
    cursor.execute("INSERT INTO users (name, age, email) VALUES ('Bob', 30, 'bob@example.com')")
except sqlite3.IntegrityError as e:
    print('Error inserting data:', e)

# Commit changes
conn.commit()


### Incerting multiple lines at a time using `executemany`.

In [ ]:
# Data to be inserted
users = [
    ('Charlie', 28, 'charlie@example.com'),
    ('David', 35, 'david@example.com'),
    ('Eve', 22, 'eve@example.com')
]

# Insert multiple rows using executemany
try:
    cursor.executemany("INSERT INTO users (name, age, email) VALUES (?, ?, ?)", users)
except sqlite3.IntegrityError as e:
    print('Error inserting data:', e)

# Commit changes
conn.commit()

Note the usage of ? as placeholders. The tuple of ?'s must match the number of columns.

In [ ]:
cursor.execute("SELECT * FROM users")
cursor.fetchall()


## Updating Data

To update existing records, we use the `UPDATE` statement.


In [17]:
# Update a user's age
cursor.execute("UPDATE users SET age = 26 WHERE email = 'alice@example.com'")

# Commit changes
conn.commit()

In [ ]:
cursor.execute("SELECT * FROM users")
cursor.fetchall()


## Deleting Data

To remove records, we use the `DELETE` statement.


In [19]:
# Delete a user
cursor.execute("DELETE FROM users WHERE name = 'Bob'")

# Commit changes
conn.commit()


## Closing the Connection

Always close the database connection when you're done to free up resources.


In [ ]:
# Close the connection
conn.close()

## Pandas and SQLite

In [21]:
conn = sqlite3.connect('sales.db')
cursor = conn.cursor()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

In [ ]:
# Read the users table into a Pandas DataFrame
df = pd.read_sql_query("SELECT * FROM Artist", conn)
df.head()


In [24]:
query = "SELECT * FROM InvoiceLine"

In [ ]:
# Execute the query
result = pd.read_sql_query(query, conn)

# Display the result
result.head()

In [26]:
query = """
SELECT InvoiceID, Track.Name AS NAME, SUM(Quantity) as TotalSold
FROM InvoiceLine
JOIN Track ON InvoiceLine.TrackID = Track.TrackID
GROUP BY Track.Name
HAVING SUM(Quantity) > 2
ORDER BY TotalSold DESC;
"""

In [ ]:
pd.read_sql_query(query, conn)

# Now it's your turn!

Below are the questions designed to explore and analyze the data in the `sales.db` database:

1. **How many records are there in the `Invoice` table?**

2. **What are the names of all the artists in the `Artist` table?**

3. **What is the total revenue generated from all invoices?**

4. **Which customer has made the most purchases?**

5. **What are the top 5 most sold tracks?**

6. **What is the average unit price of tracks sold?**

7. **How many customers are there in each country?**

8. **Which employee has generated the most revenue from their sales?**

9. **What are the top 3 genres by the number of tracks?**


### Answer Below: